# Supervised Fine-Tuning (SFT) Simple Template





Supervised fine-tuning (SFT) is a technique used to adapt a pre-trained Large Language Model (LLM) to a specific downstream task using labeled data.This process allows the model to learn task-specific patterns and nuances by adapting its parameters according to the specific data distribution and task requirements.

`I prepared this Supervised Fine-Tuning (SFT) template for my use case, but you could change it to suit your requirements.`



To View My Account:

* [Hugging Face ](https://huggingface.co/santhoshmlops)

* [Git Hub](https://github.com/santhoshmlops)

To View Some other Fine Tuning Template:

* [Fine Tuning Template ](https://github.com/santhoshmlops/MyHF_LLM_FineTuning/tree/main/FineTuningTemplate)


To View My Model Fine Tuning  NoteBook:

* [MY HF LLM Fine-Tuning](https://github.com/santhoshmlops/MyHF_LLM_FineTuning)



## Setting Up on Google Colab
Google Colab provides a convenient, cloud-based environment with access to powerful GPUs like the `T4`. If you choose Colab for this tutorial, make sure to select a GPU runtime by going to `Runtime > Change runtime type > T4 GPU`. This ensures that your notebook has access to the necessary computational resources.

## Setting Up Hugging Face Authentication

On Google Colab, you can safely store your Hugging Face token by using Colab's "Secrets" feature. This can be done by clicking on the "Key" icon in the sidebar, selecting "`Secrets`", and adding a new secret with the name `HF_TOKEN` and your Hugging Face token as the value. This method ensures that your token remains secure and is not exposed in your notebook's code.

# Step 1 - Install the required Python packages

In [ ]:
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U bitsandbytes
!pip install -q -U trl
!pip install -q -U accelerate
!pip install -q -U datasets

# Step 2 - Logging into Hugging Face Hub
Paste the Hugging Face Hub Write API KEY

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Step 3 - Loading Required Libraries

In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,DataCollatorForLanguageModeling
from peft import LoraConfig,PeftModel, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from accelerate import Accelerator

# Step 4 - Setting Model Parameters for SFT
`Note:` The parameter can be changed for fine tuning, or it can be left as it is and filled with the value of the empty parameter.

In [ ]:
# Load Model for Tuning
model_ckpt = ""  # Change the model_ckpt as your wish. For eg - "microsoft/phi-1_5"
hf_user_name = "santhoshmlops"
hub_model_ckpt = hf_user_name+"/"+ model_ckpt.split("/")[-1]+"-SFT" # Change the hub_model_ckpt as your wish. For eg - "santhoshmlops/microsoft_phi-1_5_merged-SFT"
dataset_name = ""

# Lora Parameters
r= 16
lora_alpha = 32
lora_dropout = 0.05
bias = "none"
task_type = "CAUSAL_LM"
target_modules = ["q_proj","k_proj", "v_proj","o_proj","gate_proj","up_proj","down_proj"]    # Change the Target modules based on the model for tuning For eg - ["q_proj","k_proj"]

# BitsandBytes Parameters
load_in_4bit = True
bnb_4bit_quant_type = "nf4"
bnb_4bit_compute_dtype = torch.float16
bnb_4bit_use_double_quant = True

# Automodel Parameters
device_map = {"": Accelerator().local_process_index}
torch_dtype = torch.float16

# Tokenizer Parameters
trust_remote_code = True

# Training Parameters
output_dir = model_ckpt.split("/")[-1]+"-SFT"   # Change the model_ckpt as your wish. For eg - "microsoft_phi-1_5_merged-SFT"
num_train_epochs = 1
per_device_train_batch_size = 2
gradient_accumulation_steps = 2
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.003
optim = "paged_adamw_8bit"
lr_scheduler_type = "cosine"
max_steps = 750
warmup_ratio = 0.03
group_by_length = True
save_steps = 100
save_strategy = "epoch"
logging_steps = 100
logging_dir = "./logs"
fp16 = True
bf16 = False
push_to_hub = True
neftune_noise_alpha = 5
report_to = "tensorboard"

# SFT Training Parameters
train_cln_name = "text"
packing = False
max_seq_length = 1024

# Merge and push the model to Hub
low_cpu_mem_usage = True
return_dict = True

# Step 5 - Loading and Formatting the Dataset
`Note:` Prepare your dataset for fine tuning by defining and formatting it for your use case. The `def create_data():` function is an example for tuning the dataset.

In [ ]:
def create_data():
  data = load_dataset(dataset_name, split="train")
  return data

data = create_data()
print(data[0])

# Step 6 - Fine-Tuning with Lora and Supervised Finetuning

In [ ]:
# Load the model and tokenizer with specified configurations.
tokenizer = AutoTokenizer.from_pretrained(
    model_ckpt,
    trust_remote_code=trust_remote_code
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant
)

model = AutoModelForCausalLM.from_pretrained(
    model_ckpt,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=trust_remote_code,
    torch_dtype=torch_dtype
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    max_grad_norm=max_grad_norm,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    optim=optim,
    lr_scheduler_type=lr_scheduler_type,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    save_steps=save_steps,
    save_strategy=save_strategy,
    logging_steps=logging_steps,
    logging_dir=logging_dir,
    fp16=fp16,
    bf16=bf16,
    push_to_hub=push_to_hub,
    neftune_noise_alpha = neftune_noise_alpha,
    report_to=report_to
)

# Prepare the model with LoRA (Low-Rank Adaptation) configuration.
lora_config = LoraConfig(
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias=bias,
    task_type=task_type,
    target_modules=target_modules
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Create a trainer for training the model.
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=lora_config,
    dataset_text_field=train_cln_name,
    args=training_args,
    tokenizer=tokenizer,
    packing=packing,
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Step 7 - Lets start the training process

In [ ]:
# Train the model and save it.
trainer.train()
trainer.push_to_hub()

# Step 8 - Merge the model with LoRA weights

In [ ]:
# Clear the memory footprint
del model, trainer
torch.cuda.empty_cache()

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt,
                                          trust_remote_code=trust_remote_code)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(model_ckpt,
                                                  low_cpu_mem_usage=low_cpu_mem_usage,
                                                  return_dict=return_dict,
                                                  torch_dtype=torch_dtype,
                                                  device_map=device_map,trust_remote_code=trust_remote_code)

# Merge models
merged_model = PeftModel.from_pretrained(base_model,hub_model_ckpt, from_transformers=True)
merged_model = merged_model.merge_and_unload()

# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(hub_model_ckpt, use_temp_dir=False)
tokenizer.push_to_hub(hub_model_ckpt, use_temp_dir=False)

# Step 8 - Inferencing with the model output

In [ ]:
# pip install bitsandbytes accelerate
import time
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained("santhoshmlops/gemma-2b-it-SFT")
model = AutoModelForCausalLM.from_pretrained("santhoshmlops/gemma-2b-it-SFT", quantization_config=quantization_config)

st_time = time.time()

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids,max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

print(time.time()-st_time)